# dahyun+darwin = dahwin

In [ ]:
import torch
from torch import *
import matplotlib.pyplot as plt
from sklearn.datasets import *
from torch import nn
from sklearn import datasets
# x = dir(datasets)
# print("\n".join(x))

# Make calssifiction data and get it ready

In [ ]:
# make 1000 saples
n_sample = 1000

# create circles
x,y = make_circles(n_sample,
                    noise=0.03,
                   random_state=42
                    )

In [ ]:
len(x),len(y)

In [ ]:
print(f'First 5 samples of x:\n{x[:10]}')
print(f"First 5 samples of y:\n{y[:10]}")

In [ ]:
# Make DataFarame of circle data
import pandas as pd
circles = pd.DataFrame({'x1':x[:,0],
                       'x2':x[:,1],
                       'label':y})
circles.head(10)

In [ ]:
plt.scatter(x=x[:,0],
              y=x[:,1],
              c= y,
              cmap=plt.cm.RdYlBu)

# **1.1 Check input and output shapes**

In [ ]:
x.shape,y.shape

In [ ]:
y[0]

In [ ]:
x

In [ ]:
# view the first example of features and labels
x_sample = x[0]
y_sample = y[0]
print(f"values for one sample of x:{x_sample} and the same for y:{y_sample}")
print(f"shapes for one sample of x:{x_sample.shape} and the same for y:{y_sample.shape}")

# 1.2 Turn data into tensors and create train and test splits

In [ ]:
type(x)

In [ ]:
x = torch.from_numpy(x).type(torch.float)
y = torch.from_numpy(y).type(torch.float)
x[:5] , y[:5]

In [ ]:
type(x)

In [ ]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible

len(x_train), len(x_test), len(y_train), len(y_test)

# **2. Building a model**

Let's build a model to clssify our blue and red dots.
To do so , we wan to :
1. Setup device agnoistic
2. Construct a model (by subclassing nn.Module)
3. Define a loss fuction and optimizer
4. Create a trainng and test loop

In [ ]:
# make device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Now we've setup device agonstic code , let's create a model that:
1. subclass nn.Modeule(almost all models in pytorch subclass nn.modeule)
2. create 2 nn.Linear() layers that are capable of handiling the shapes of our data
3. defines a forward() method that outlilnes the foirward pass (or forwrd computation) 0f the model
4. Instatite an instance of our model calss and send it to the target device

In [ ]:
x_train.shape

In [ ]:
y_train[0]

In [ ]:
# 1. costruct a model that subclass nn.Module
class CircleModelV0(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1= nn.Linear(in_features=2,out_features=5)
    self.layer_2 = nn.Linear(in_features=5,out_features=1)
  def forward(self,x):
    return self.layer_2(self.layer_1(x)) # x-> layer_1 -> output

model_0 = CircleModelV0().to(device)
model_0

In [ ]:
# Let's replicate the model above using nn.Sequential()
model_0 = nn.Sequential(
    nn.Linear(in_features= 2,out_features=5,),
    nn.Linear(in_features=5,out_features=1)
).to(device)
model_0

In [ ]:
model_0.state_dict()

In [ ]:
with torch.inference_mode():

    untrained_pre = model_0(x_test.to(device))
print(f'Length of predictions:{len(untrained_pre)},shape:{untrained_pre.shape}')
print(f"Length of test samples:{len(x_test)},shape:{x_test.shape}")
print(f"\nFirst 10 predictions:\n{torch.round(untrained_pre[:10])}")
print(f"\nFirst 10 labels:\n{y_test[:10]}")

In [ ]:
x_test[:10],y_test[:10]

# set up
but which one should we use?
For example for regression you might want to use MAE or MSE ( mean absolute error or mean squard error)

For example for classicfication you might want to use binary cross entropy or categorical cross entropy (cross entropy)

As a reminder , the loss fucion mesures how wrong you model's predictions are



*   For some common choices of loss fuction and optimizer : https://www.learnpytorch.io/02_pytorch_classification/#21-setup-loss-function-and-optimizer

*   For the loss fuction we're going to use torch.nn.BECWithLogitaLoss(),for more on what binary coross entropy(BCE) is
*  For a definition of logit in deep learning


*   For different optimizers see torch.optim






In [ ]:
# Create a loss function
# loss_fn = nn.BCELoss() # BCELoss = no sigmoid built-in
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), 
                            lr=0.1)


In [ ]:
model_0.state_dict()

In [ ]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

# 3. Train Model
To train our model , we're going to need to build a training lop with the following steps
1. Forward
2. Calulate the loss
3. Optimizer zero grad
4. Loss backword (backpropagation)
5. Opimizer stp (gradient desent)


### 3.1 Going from raw logits-> prediction probabilities -> prediction labels
our model ouputs are going to be raw **logits**

we can convert these **logits** into prediction probabilites by passing them 
to some kinde of activation fuction(e.g  sigmoid for binary classificaiton and softmax for multicalss classification)
Then we can convert our model's prediction probabilities to prediction labels by either rounding them or taking the argmax()

In [ ]:
# View the first 5 ouputs of the forward pas on the test data
model_0.eval()
with torch.inference_mode():
  y_logits = model_0(x_test.to(device))[:5]
y_logits

In [ ]:
y_test[:5]

In [ ]:
# Use the sigmoid activation fuction on our model logits to turn them into prediction probabilities
y_pred_pro = torch.sigmoid(y_logits)
y_pred_pro

# For our prediction probability values, we need to perform a range style rounding on them:


*   y_pred_pro >=0.5 y=1(class 1)
*   y_pred_pro <=0 , y=0 (class 0)




In [ ]:
# Find the predicted labels 
y_pre = torch.round(y_pred_pro)
y_pre_labels = torch.round(torch.sigmoid(model_0(x_test.to(device))[:5]))
# check for equalitiy
print(torch.eq(y_pre.squeeze(),y_pre_labels.squeeze()))
y_pre.squeeze()

In [ ]:
torch.manual_seed(42)

# Set the number of epochs


epochs = 10  # Define the number of epochs to run

# Put data to target device
X_train, y_train = x_train.to(device), y_train.to(device)
X_test, y_test = x_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in range(0,100):
    ### Training
    model_0.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model_0(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
    y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
  
    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_0(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")



# 4 Make predictions and evalutae the model
From the metrics it looks like our model is random guessing.


How could we investigate this futher?

I've got an idea.

The data explorer's motto!

'Visualize , visualize, visualize!'

Let's make a plot of our model's predictions, the data it's tryring to predict on and the decision boundary

it's creating for whether something is class 0 or class 1.
To do so, we'll write some code to download and import the helper_fuctions.py from the Learn pytorch for deep learning repo.

it contains a helful fuction called plot_decision_boundary() which creates a numpy meshgrid to visually plot the different points where our model is predictiong cretain classes

We'll also import plot_predictions() which we wrote in notebook 01 to use later

In [ ]:
import requests
from pathlib import Path
# Dowload helper fuctions from learn PYtorch repo(if not already downloded)

if Path('helper_functions.py').is_file():
  print('helper_functions.py already exists')
else:
  print('Downloading helper_fuctions.py')
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open('helper_functions.py','wb') as f:
    f.write(request.content)
from helper_functions import plot_predictions, plot_decision_boundary


In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title('Train')
plot_decision_boundary(model_0,x_train,y_train)
plt.subplot(1,2,2)
plt.title('test')
plot_decision_boundary(model_0,x_test,y_test)

# 5. Improving a model (from a model perpective)

In [ ]:
model_0.state_dict()

# 5. Improving a moel ( from a model persprective)
* Add more layers = give the model more chances to learn about patterns in the data
* Add more hidden units - go from 5 hidden units to 10 hidden units
* Fit for longer
* Changing the activation functions
* Change the learning rate
* Change the loss fuction
These options are all from a model's perpective because they deal dirctly with the model, rather than the data.
And because these options are all values we (as machine learning engineers and data scientists) can change, they are referred as **hyperparameters**

Let's try and improve our model by:
* Adding more hidden units: 5->10
* Increase the numbmer of layers: 2 ->3
* Increase the number of epochs: 100 ->1000

In [ ]:
class CircleModelV1(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=2,out_features=10)
    self.layer_2 = nn.Linear(in_features=10,out_features=10)
    self.layer_3 = nn.Linear(in_features=10,out_features=1)
  def forward(self,x):
      # z = self.layer_1(x)
      # z = self.layer_2(x)
      # z = self.layer_3(x)
      return self.layer_3(self.layer_2(self.layer_1(x))) # this way of wiriting operarations levarages speed ups where possible behind the)
model_1 = CircleModelV1().to(device)
model_1

In [ ]:
model_1.state_dict()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params= model_1.parameters(),lr=0.1)

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
epoch = 1000
x_train,y_train = x_train.to(device),y_train.to(device)
for epoch in range(0,epochs):
  model_1.train()
  y_logits = model_1(x_train).squeeze()
  y_pre = torch.round(torch.sigmoid(y_logits))
  loss = loss_fn(y_logits,y_train)
  acc = accuracy_fn(y_train,y_pre)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  model_1.eval()
  with torch.inference_mode():
    test_logits = model_1(x_test).squeeze()
    test_pre = torch.round(torch.sigmoid(test_logits))
    test_loss = loss_fn(test_logits,y_test)
    test_acc = accuracy_fn(y_test,test_pre)
  # Print out what's happening every 10 epochs
  if epoch % 100 == 0:
     print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

In [ ]:
# First decision boundary of the model
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title('Treain')
plot_decision_boundary(model_1,x_train,y_train)
plt.subplot(1,2,2)
plt.title('Test')
plot_decision_boundary(model_1,x_test,y_test)

In [ ]:
weight = 0.7
bias = 0.3
start = 0
end = 1
step = 0.01
# crate daa 
x_regression = torch.arange(start,end,step).unsqueeze(dim=1)
y_regression = weight * x_regression + bias
print(len(x_regression))
x_regression[:5],y_regression[:5]

# create train and test splits

In [ ]:
# Create train and test splits
train_split = int(0.8 * len(x_regression)) # 80% of data used for training set
X_train_regression, y_train_regression = x_regression[:train_split], y_regression[:train_split]
X_test_regression, y_test_regression = x_regression[train_split:], y_regression[train_split:]

# Check the lengths of each split
print(len(X_train_regression), 
    len(y_train_regression), 
    len(X_test_regression), 
    len(y_test_regression))

In [ ]:
plot_predictions(X_train_regression,y_train_regression,X_test_regression,y_test_regression)

# 5.2 Adjusting model_1 to fit a straight line
Now we've got some data , let's recreate modelL_1 but with a loss fucntion suted to our regression data

In [ ]:
# ame architecture as model_1 (but using nn.Sequential)
model_2 = nn.Sequential(
    nn.Linear(in_features=1,out_features=10),
    nn.Linear(in_features=10,out_features=10),
    nn.Linear(in_features=10,out_features=1)
).to(device)
model_2

# We'll setup the loss function to be nn.L1Loss() (the same as mean absolute error) and the optimizer to be torch.optim.SGD().

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model_2.parameters(),lr=0.1)


In [ ]:
# Train the model
torch.manual_seed(42)

# Set the number of epochs
epochs = 1000

# Put data to target device
X_train_regression, y_train_regression = X_train_regression.to(device), y_train_regression.to(device)
X_test_regression, y_test_regression = X_test_regression.to(device), y_test_regression.to(device)

for epoch in range(0,epochs):
    ### Training 
    # 1. Forward pass
    y_pred = model_2(X_train_regression)
    
    # 2. Calculate loss (no accuracy since it's a regression problem, not classification)
    loss = loss_fn(y_pred, y_train_regression)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_2.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_pred = model_2(X_test_regression)
      # 2. Calculate the loss 
      test_loss = loss_fn(test_pred, y_test_regression)

    # Print out what's happening
    if epoch % 100 == 0: 
        print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Test loss: {test_loss:.5f}")

#Okay, unlike model_1 on the classification data, it looks like model_2's loss is actually going down.

Let's plot its predictions to see if that's so.

And remember, since our model and data are using the target device, and this device may be a GPU, however, our plotting function uses matplotlib and matplotlib can't handle data on the GPU.

To handle that, we'll send all of our data to the CPU using .cpu() when we pass it to

In [ ]:
# Turn on evaluation mode
model_2.eval()

# Make predictions (inference)
with torch.inference_mode():
    y_preds = model_2(X_test_regression)

# Plot data and predictions with data on the CPU (matplotlib can't handle data on the GPU)
# (try removing .cpu() from one of the below and see what happens)
plot_predictions(train_data=X_train_regression.cpu(),
                 train_labels=y_train_regression.cpu(),
                 test_data=X_test_regression.cpu(),
                 test_labels=y_test_regression.cpu(),
                 predictions=y_preds.cpu());

# 6. The missing piece: non-linearity
We've seen our model can draw straight (linear) lines, thanks to its linear layers.

But how about we give it the capacity to draw non-straight (non-linear) lines?

How?

Let's find out.

# 6.1 Recreating non-linear data (red and blue circles)
First, let's recreate the data to start off fresh. We'll use the same setup as before.

In [ ]:
# Make and plot data
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles

n_samples = 1000

X, y = make_circles(n_samples=1000,
    noise=0.03,
    random_state=42,
)

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdBu);

In [ ]:
# Convert to tensors and split into train and test sets
import torch
from sklearn.model_selection import train_test_split

# Turn data into tensors
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2,
                                                    random_state=42
)

X_train[:5], y_train[:5]

# 6.2 Building a model with non -linearity
Now here comes the fun part


*   Linear = straight line
*   non-linear = non-straight lines
# Artificial nerural network are a large combinaation of lineawr and non-linear fuctions which are potentially abel to find patterns in data


In [ ]:
# Build model with non-linear activation function
from torch import nn
class CircleModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=2, out_features=10)
        self.layer_2 = nn.Linear(in_features=10, out_features=10)
        self.layer_3 = nn.Linear(in_features=10, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model 
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_3 = CircleModelV2().to(device)
print(model_3)

In [ ]:
# Setup loss and optimizer 
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model_3.parameters(), lr=0.1)

#  6.3 train model in non-linearity

In [ ]:
# random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)
x_train,y_train = x_train.to(device),y_train.to(device)
x_test,y_test = x_test.to(device),y_test.to(device)
epochs = 1000
for epoch in range(0,epochs):
  # train
  model_3.train()\
  # 1.Forward pass
  y_logits = model_3(x_train).squeeze()
  y_pre = torch.round(torch.sigmoid(y_logits))
  # 2. cALCULATEL THE LOSS
  loss = loss_fn(y_logits,y_train)
  acc = accuracy_fn(y_train,y_pre)
  # 3. Optimizer zero grad
  optimizer.zero_grad()
  # 4.Loss backward
  loss.backward()
  # 5. step the optimizer
  optimizer.step()
  ### Testing 
  model_3.eval()
  with torch.inference_mode():
    test_logits = model_3(x_test).squeeze()
    test_pre = torch.round(torch.sigmoid(test_logits))
    test_loss = loss_fn(test_logits,y_test)
    test_acc = accuracy_fn(y_test,test_pre)
  if epoch % 100 == 0:
    
    print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")


In [ ]:
# Make predictions 
model_3.eval()
with torch.inference_mode():
  y_pre = torch.round(torch.sigmoid(model_3(x_test))).squeeze()
y_pre[:10],y_test[:10]

In [ ]:
# Plot decision boundaries 
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title('Train')
plot_decision_boundary(model_1,x_train,y_train)
plt.subplot(1,2,2)
plt.title('Test')
plot_decision_boundary(model_3,x_test,y_test)

## 7. Replication non-linear activation functioins
Neural netwoks , rather than us telllilng the model what to learn we give it the tools to discovr patterns in data and it tries to figure out the patters on its own
An these tools are linear and non-linear fuctions

In [ ]:
# create a tensor
a = torch.arange(-10,10,1)
a.dtype

In [ ]:
a

In [ ]:
#visualize the tensor
plt.plot(a)

In [ ]:
plt.plot(torch.relu(a))

In [ ]:
def myrelu(a):
  return torch.maximum(torch.tensor(0),x) 
myrelu(a)

In [ ]:
# plot myrelu activation fuction
plt.plot(torch.relu(a))

In [ ]:
# Now let's do the same for sigmoid 
def mysigmoid(x):
  return 1/1 + torch.exp(-x)

In [ ]:
plt.plot(torch.sigmoid(a))

In [ ]:
plt.plot(sigmoid(a))

# 8. Putting it all together with a multiclass classificaation problem


*   Binary classification = one thing or another ( cat vs dog, spam or not , fraud or not froud )
*   Multi-class  classification = more than thing or another (cat vs dog vs chicken)



# Creating a toy multi-class detasets


In [ ]:
from sklearn.datasets import make_blobs
# set hyperparameter for data   creations
num_classes = 4 
num_features = 2
random_seed = 42
# create multiclass data
x_blob ,y_blob = make_blobs(n_samples=1000,n_features=num_features,centers=num_classes, cluster_std=1.5,random_state=random_seed)
x_blob = torch.from_numpy(x_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)
x_blob_train,x_blob_test,y_blob_train,y_blob_test = train_test_split(x_blob,y_blob,test_size=0.2,random_state=random_seed)
# plot
plt.figure(figsize=(10, 7))
plt.scatter(x_blob[:, 0], x_blob[:, 1], c=y_blob, cmap=plt.cm.RdYlBu);

# Building a multi class classification model in Pytorch

In [ ]:
class DawinModel(nn.Module):
  def __init__(self, input_features, output_features, hidden_units=8):
    super().__init__()
    self.linear_layer_stack = nn.Sequential(
        nn.Linear(in_features=input_features, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_features)
    )
    
  def forward(self, x):
    return self.linear_layer_stack(x)
# Create an instance of DawinModel and send it to teh target device
model_4 = DawinModel(input_features=2,output_features=4,hidden_units=8).to(device)
model_4

# create loss_fn , optimizer and traning loop for this multi class classification model

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params= model_4.parameters(),lr=0.1)


# Getting prediction prbabilities for a multi-class pytorch model 
In oder to evaluate and  train and test our model, w eeee need to convert our model's outputs (logits) to prediction prebabilities and then to prediction labels .
Logits -pre - pre labels

In [ ]:
model_4.eval()
with torch.inference_mode():
    y_logits = model_4(x_blob_test.to(device))
y_logits[:10]

In [ ]:
# converts our model's logit outputs to prediction probabilities
y_pre = torch.softmax(y_logits,dim=1)
print(y_logits[:5])
print(y_pre[:5])

In [ ]:
torch.sum(y_pre[0])

In [ ]:
torch.argmax(y_pre[0])

In [ ]:
y_pre1 = torch.argmax(y_pre,dim=1)
y_pre1

In [ ]:
y_blob_train.shape

# Creating  a traning loop and testing loop for multi class classifiction model

In [ ]:
# FIt the multiclass model to the data
torch.manual_seed(42)
torch.cuda.manual_seed(42)
epochs = 100
# Put data to target device
x_blob_train, y_blob_train = x_blob_train.to(device), y_blob_train.to(device)
x_blob_test, y_blob_test = x_blob_test.to(device), y_blob_test.to(device)
for epoch in range(0,epochs):
  model_4.train()
  # 1. Forward pass
  y_logits = model_4(x_blob_train)
  y_pre = torch.softmax(y_logits,dim=1).argmax(dim=1)
  loss = loss_fn(y_logits,y_blob_train)
  acc = accuracy_fn(y_blob_train,y_pre)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  ### Testing
  model_4.eval()
  with torch.inference_mode():
    test_logits = model_4(x_blob_test)
    test_pre = torch.softmax(test_logits,dim=1).argmax(dim=1)
    test_loss = loss_fn(test_logits,y_blob_test)
    test_acc = accuracy_fn(y_blob_test,test_pre)
  if epoch % 10 == 0:
    print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")


In [ ]:
# Make predictions 
model_4.eval()
with torch.inference_mode():
  y_logits = model_4(x_blob_test)
y_logits[:10]

Alright, looks like our model's predictions are still in logit form.

Though to evaluate them, they'll have to be in the same form as our labels (`y_blob_test`) which are in integer form.

Let's convert our model's prediction logits to prediction probabilities (using `torch.softmax()`) then to prediction labels (by taking the `argmax()` of each sample).

> **Note:** It's possible to skip the `torch.softmax()` function and go straight from `predicted logits -> predicted labels` by calling `torch.argmax()` directly on the logits.
>
> For example, `y_preds = torch.argmax(y_logits, dim=1)`, this saves a computation step (no `torch.softmax()`) but results in no prediction probabilities being available to use. 

In [ ]:
y_pre_pro = torch.softmax(y_logits,dim=1)
y_pre = y_pre_pro.argmax(dim=1)
print(f"Predictions: {y_pre[:10]}\nLabels: {y_blob_test[:10]}")
print(f"Test accuracy: {accuracy_fn(y_true=y_blob_test, y_pred=y_preds)}%")

Nice! Our model predictions are now in the same form as our test labels.

Let's visualize them with plot_decision_boundary(), remember because our data is on the GPU, we'll have to move it to the CPU for use with matplotlib (plot_decision_boundary() does this automatically for us).

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Train")
plot_decision_boundary(model_4,x_blob_train,y_blob_train)
plt.subplot(1,2,2)
plot_decision_boundary(model_4,x_blob_test,y_blob_test)

In [ ]:
try:
    from torchmetrics import Accuracy
except:
    !pip install torchmetrics # this is the version we're using in this notebook (later versions exist here: https://torchmetrics.readthedocs.io/en/stable/generated/CHANGELOG.html#changelog)
    from torchmetrics import Accuracy

# Setup metric and make sure it's on the target device
torchmetrics_accuracy = Accuracy(task='multiclass', num_classes=4).to(device)

# Calculate accuracy
torchmetrics_accuracy(y_pre, y_blob_test)